In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive -o nonempty

In [0]:
import os
os.chdir("drive/hackerearth")

# Importing the libraries

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.cross_validation import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge
from sklearn.metrics import f1_score
import math

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Reading the files

In [0]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
building_use=pd.read_csv('Building_Ownership_Use.csv')
building_structure=pd.read_csv('Building_Structure.csv')

# Merging all the files into a single file

In [0]:
k=pd.merge(building_structure,building_use,on='building_id')
train_df=pd.merge(train_df,k,on='building_id')
test_df=pd.merge(test_df,k,on='building_id')

# Data Preprocessing

Giving numerical values to **Damage grade**

In [0]:
# Damage Grade
p={'Grade 1':0,'Grade 2':1,'Grade 3':2,'Grade 4':3,'Grade 5':4}
train_df['damage_grade']=train_df['damage_grade'].map(p)

Filling the **Null Values**

In [0]:
# Has repair started
train_df['has_repair_started'].fillna(0.0,inplace=True)
test_df['has_repair_started'].fillna(0.0,inplace=True)

# Count Families
train_df['count_families'].fillna(1.0,inplace=True)
test_df['count_families'].fillna(1.0,inplace=True)

# Feature Engineering

Making new features by **Pairing two  or more columns**

In [0]:
# Age of building Vdcmun id and mud mortar superstructure wise
k=train_df.groupby(['vdcmun_id','has_superstructure_mud_mortar_stone'])['age_building'].mean().reset_index()
k=k.rename(columns={'age_building':'age_building_new'})
train_df=train_df.merge(k,on=['vdcmun_id','has_superstructure_mud_mortar_stone'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_superstructure_mud_mortar_stone'],how='left')

# Age of buildings where repair has been started
k=train_df.groupby(['vdcmun_id','has_repair_started'])['age_building'].mean().reset_index()
k=k.rename(columns={'age_building':'age_building_new_2'})
train_df=train_df.merge(k,on=['vdcmun_id','has_repair_started'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_repair_started'],how='left')

# Age of buildings which are at geotechnical risk
k=train_df.groupby(['vdcmun_id','has_geotechnical_risk'])['age_building'].mean().reset_index()
k=k.rename(columns={'age_building':'age_building_new_3'})
train_df=train_df.merge(k,on=['vdcmun_id','has_geotechnical_risk'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_geotechnical_risk'],how='left')

# Age of building Vdcmun id and has secondary use wise
k=train_df.groupby(['vdcmun_id','has_secondary_use'])['age_building'].mean().reset_index()
k=k.rename(columns={'age_building':'age_building_new_4'})
train_df=train_df.merge(k,on=['vdcmun_id','has_secondary_use'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_secondary_use'],how='left')

# Age of buildings which are at geotechnical risk of fault cracks
k=train_df.groupby(['vdcmun_id','has_geotechnical_risk_fault_crack'])['age_building'].mean().reset_index()
k=k.rename(columns={'age_building':'age_building_new_5'})
train_df=train_df.merge(k,on=['vdcmun_id','has_geotechnical_risk_fault_crack'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_geotechnical_risk_fault_crack'],how='left')

# Pre heights of bulidings which are having mud mortar superstructure
k=train_df.groupby(['vdcmun_id','has_superstructure_mud_mortar_stone'])['height_ft_pre_eq'].mean().reset_index()
k=k.rename(columns={'height_ft_pre_eq':'pre_height_new'})
train_df=train_df.merge(k,on=['vdcmun_id','has_superstructure_mud_mortar_stone'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_superstructure_mud_mortar_stone'],how='left')

# Pre heights of buildings which are at geotechnical risk
k=train_df.groupby(['vdcmun_id','has_geotechnical_risk'])['height_ft_pre_eq'].mean().reset_index()
k=k.rename(columns={'height_ft_pre_eq':'pre_height_new_2'})
train_df=train_df.merge(k,on=['vdcmun_id','has_geotechnical_risk'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_geotechnical_risk'],how='left')

# Pre heights where repairs has been started
k=train_df.groupby(['vdcmun_id','has_repair_started'])['height_ft_pre_eq'].mean().reset_index()
k=k.rename(columns={'height_ft_pre_eq':'pre_height_new_3'})
train_df=train_df.merge(k,on=['vdcmun_id','has_repair_started'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_repair_started'],how='left')

# Pre heights of bulidings which are in secondary use
k=train_df.groupby(['vdcmun_id','has_secondary_use'])['height_ft_pre_eq'].mean().reset_index()
k=k.rename(columns={'height_ft_pre_eq':'pre_height_new_4'})
train_df=train_df.merge(k,on=['vdcmun_id','has_secondary_use'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_secondary_use'],how='left')

# Pre heights of buildings which are at geotechnical risk of fault cracks
k=train_df.groupby(['vdcmun_id','has_geotechnical_risk_fault_crack'])['height_ft_pre_eq'].mean().reset_index()
k=k.rename(columns={'height_ft_pre_eq':'pre_height_new_5'})
train_df=train_df.merge(k,on=['vdcmun_id','has_geotechnical_risk_fault_crack'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_geotechnical_risk_fault_crack'],how='left')

# Post heights of bulidings which are having mud mortar superstructure
k=train_df.groupby(['vdcmun_id','has_superstructure_mud_mortar_stone'])['height_ft_post_eq'].mean().reset_index()
k=k.rename(columns={'height_ft_post_eq':'post_height_new'})
train_df=train_df.merge(k,on=['vdcmun_id','has_superstructure_mud_mortar_stone'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_superstructure_mud_mortar_stone'],how='left')

# Post heights of buildings which are at geotechnical risk
k=train_df.groupby(['vdcmun_id','has_geotechnical_risk'])['height_ft_post_eq'].mean().reset_index()
k=k.rename(columns={'height_ft_post_eq':'post_height_new_2'})
train_df=train_df.merge(k,on=['vdcmun_id','has_geotechnical_risk'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_geotechnical_risk'],how='left')

# Post heights where repairs has been started
k=train_df.groupby(['vdcmun_id','has_repair_started'])['height_ft_post_eq'].mean().reset_index()
k=k.rename(columns={'height_ft_post_eq':'post_height_new_3'})
train_df=train_df.merge(k,on=['vdcmun_id','has_repair_started'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_repair_started'],how='left')

# Post heights of bulidings which are in secondary use
k=train_df.groupby(['vdcmun_id','has_secondary_use'])['height_ft_post_eq'].mean().reset_index()
k=k.rename(columns={'height_ft_post_eq':'post_height_new_4'})
train_df=train_df.merge(k,on=['vdcmun_id','has_secondary_use'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_secondary_use'],how='left')

# Post heights of buildings which are at geotechnical risk of fault cracks
k=train_df.groupby(['vdcmun_id','has_geotechnical_risk_fault_crack'])['height_ft_post_eq'].mean().reset_index()
k=k.rename(columns={'height_ft_post_eq':'post_height_new_5'})
train_df=train_df.merge(k,on=['vdcmun_id','has_geotechnical_risk_fault_crack'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_geotechnical_risk_fault_crack'],how='left')

# Pre count floors of bulidings which are having mud mortar superstructure
k=train_df.groupby(['vdcmun_id','has_superstructure_mud_mortar_stone'])['count_floors_pre_eq'].mean().reset_index()
k=k.rename(columns={'count_floors_pre_eq':'pre_count_floor_new'})
train_df=train_df.merge(k,on=['vdcmun_id','has_superstructure_mud_mortar_stone'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_superstructure_mud_mortar_stone'],how='left')

# Pre count floors of buildings which are at geotechnical risk
k=train_df.groupby(['vdcmun_id','has_geotechnical_risk'])['count_floors_pre_eq'].mean().reset_index()
k=k.rename(columns={'count_floors_pre_eq':'pre_count_floor_new_2'})
train_df=train_df.merge(k,on=['vdcmun_id','has_geotechnical_risk'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_geotechnical_risk'],how='left')

# Pre count floors where repairs has been started
k=train_df.groupby(['vdcmun_id','has_repair_started'])['count_floors_pre_eq'].mean().reset_index()
k=k.rename(columns={'count_floors_pre_eq':'pre_count_floor_new_3'})
train_df=train_df.merge(k,on=['vdcmun_id','has_repair_started'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_repair_started'],how='left')

# Pre count floors of bulidings which are in secondary use
k=train_df.groupby(['vdcmun_id','has_secondary_use'])['count_floors_pre_eq'].mean().reset_index()
k=k.rename(columns={'count_floors_pre_eq':'pre_count_floor_new_4'})
train_df=train_df.merge(k,on=['vdcmun_id','has_secondary_use'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_secondary_use'],how='left')

# Pre count floors of buildings which are at geotechnical risk of fault cracks
k=train_df.groupby(['vdcmun_id','has_geotechnical_risk_fault_crack'])['count_floors_pre_eq'].mean().reset_index()
k=k.rename(columns={'count_floors_pre_eq':'pre_count_floor_new_5'})
train_df=train_df.merge(k,on=['vdcmun_id','has_geotechnical_risk_fault_crack'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_geotechnical_risk_fault_crack'],how='left')

# Post count floors of bulidings which are having mud mortar superstructure
k=train_df.groupby(['vdcmun_id','has_superstructure_mud_mortar_stone'])['count_floors_post_eq'].mean().reset_index()
k=k.rename(columns={'count_floors_post_eq':'post_count_floor_new'})
train_df=train_df.merge(k,on=['vdcmun_id','has_superstructure_mud_mortar_stone'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_superstructure_mud_mortar_stone'],how='left')

# Post count floors of buildings which are at geotechnical risk
k=train_df.groupby(['vdcmun_id','has_geotechnical_risk'])['count_floors_post_eq'].mean().reset_index()
k=k.rename(columns={'count_floors_post_eq':'post_count_floor_new_2'})
train_df=train_df.merge(k,on=['vdcmun_id','has_geotechnical_risk'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_geotechnical_risk'],how='left')

# Post count floors where repairs has been started
k=train_df.groupby(['vdcmun_id','has_repair_started'])['count_floors_post_eq'].mean().reset_index()
k=k.rename(columns={'count_floors_post_eq':'post_count_floor_new_3'})
train_df=train_df.merge(k,on=['vdcmun_id','has_repair_started'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_repair_started'],how='left')

# Post count floors of bulidings which are in secondary use
k=train_df.groupby(['vdcmun_id','has_secondary_use'])['count_floors_post_eq'].mean().reset_index()
k=k.rename(columns={'count_floors_post_eq':'post_count_floor_new_4'})
train_df=train_df.merge(k,on=['vdcmun_id','has_secondary_use'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_secondary_use'],how='left')

# Post count floors of buildings which are at geotechnical risk of fault cracks
k=train_df.groupby(['vdcmun_id','has_geotechnical_risk_fault_crack'])['count_floors_post_eq'].mean().reset_index()
k=k.rename(columns={'count_floors_post_eq':'post_count_floor_new_5'})
train_df=train_df.merge(k,on=['vdcmun_id','has_geotechnical_risk_fault_crack'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_geotechnical_risk_fault_crack'],how='left')

# Plinth area of bulidings which are having mud mortar superstructure
k=train_df.groupby(['vdcmun_id','has_superstructure_mud_mortar_stone'])['plinth_area_sq_ft'].mean().reset_index()
k=k.rename(columns={'plinth_area_sq_ft':'plinth_area_new'})
train_df=train_df.merge(k,on=['vdcmun_id','has_superstructure_mud_mortar_stone'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_superstructure_mud_mortar_stone'],how='left')

# Plinth area of buildings which are at geotechnical risk
k=train_df.groupby(['vdcmun_id','has_geotechnical_risk'])['plinth_area_sq_ft'].mean().reset_index()
k=k.rename(columns={'plinth_area_sq_ft':'plinth_area_new_2'})
train_df=train_df.merge(k,on=['vdcmun_id','has_geotechnical_risk'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_geotechnical_risk'],how='left')

# Plinth area where repairs has been started
k=train_df.groupby(['vdcmun_id','has_repair_started'])['plinth_area_sq_ft'].mean().reset_index()
k=k.rename(columns={'plinth_area_sq_ft':'plinth_area_new_3'})
train_df=train_df.merge(k,on=['vdcmun_id','has_repair_started'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_repair_started'],how='left')

# Plinth area where repairs has been started
k=train_df.groupby(['vdcmun_id','has_secondary_use'])['plinth_area_sq_ft'].mean().reset_index()
k=k.rename(columns={'plinth_area_sq_ft':'plinth_area_new_4'})
train_df=train_df.merge(k,on=['vdcmun_id','has_secondary_use'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_secondary_use'],how='left')

# Plinth area of buildings which are at geotechnical risk of fault cracks
k=train_df.groupby(['vdcmun_id','has_geotechnical_risk_fault_crack'])['plinth_area_sq_ft'].mean().reset_index()
k=k.rename(columns={'plinth_area_sq_ft':'plinth_area_new_5'})
train_df=train_df.merge(k,on=['vdcmun_id','has_geotechnical_risk_fault_crack'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','has_geotechnical_risk_fault_crack'],how='left')

Generating new columns by **Putting some conditions**

In [0]:
a=train_df['plinth_area_sq_ft']
b=test_df['plinth_area_sq_ft']

# Condition of buildings post earthquake whose age is more than average age
k=train_df.groupby(['vdcmun_id','condition_post_eq'])['age_building'].mean().reset_index()
k=k.rename(columns={'age_building':'condition-wise_age'})
train_df=train_df.merge(k,on=['vdcmun_id','condition_post_eq'],how='left')
test_df=test_df.merge(k,on=['vdcmun_id','condition_post_eq'],how='left')

# Condition of buildings post earthquake grouped with count floors
## k=train_df.groupby(['vdcmun_id','condition_post_eq'])['count_floors_pre_eq'].mean().reset_index()
## k=k.rename(columns={'count_floors_pre_eq':'condition-wise_pre_count_floors'})
## train_df=train_df.merge(k,on=['vdcmun_id','condition_post_eq'],how='left')
## test_df=test_df.merge(k,on=['vdcmun_id','condition_post_eq'],how='left')

# Condition of buildings post earthquake whose post height is more than average height
## k=train_df.groupby(['vdcmun_id','condition_post_eq'])['height_ft_post_eq'].mean().reset_index()
## k=k.rename(columns={'height_ft_post_eq':'condition-wise_post_height'})
## train_df=train_df.merge(k,on=['vdcmun_id','condition_post_eq'],how='left')
## test_df=test_df.merge(k,on=['vdcmun_id','condition_post_eq'],how='left')

# Creating difference in height as column
train_df['difference_in_height']=train_df['height_ft_pre_eq']-train_df['height_ft_post_eq']
test_df['difference_in_height']=test_df['height_ft_pre_eq']-test_df['height_ft_post_eq']

# Creating difference in floors as column
train_df['difference_in_floors']=train_df['count_floors_pre_eq']-train_df['count_floors_post_eq']
test_df['difference_in_floors']=test_df['count_floors_pre_eq']-test_df['count_floors_post_eq']

# Buildings which require attention
train_df.loc[(train_df['condition_post_eq'] == 'Not damaged') & (train_df['age_building'] >= 27.8055) & (train_df['has_repair_started'] == 0),'buildings_require_attention']=1
test_df.loc[(test_df['condition_post_eq'] == 'Not damaged') & (test_df['age_building'] >= 27.8055) & (test_df['has_repair_started'] == 0),'buildings_require_attention']=1
train_df['buildings_require_attention'].fillna(0,inplace=True)
test_df['buildings_require_attention'].fillna(0,inplace=True)

# Buildings which require immediate attention
train_df.loc[(train_df['condition_post_eq'] == 'Not damaged') & (train_df['height_ft_pre_eq'] >= 16.3089) & (train_df['has_repair_started'] == 0),'buildings_require_immediate_attention']=1
test_df.loc[(test_df['condition_post_eq'] == 'Not damaged') & (test_df['height_ft_pre_eq'] >= 16.3089) & (test_df['has_repair_started'] == 0),'buildings_require_immediate_attention']=1
train_df['buildings_require_immediate_attention'].fillna(0,inplace=True)
test_df['buildings_require_immediate_attention'].fillna(0,inplace=True)

# These Buildings also require immediate attention
train_df.loc[(train_df['condition_post_eq'] == 'Not damaged') & (train_df['count_floors_pre_eq'] >= 2.1309) & (train_df['has_repair_started'] == 0),'buildings_require_immediate_attention_2']=1
test_df.loc[(test_df['condition_post_eq'] == 'Not damaged') & (test_df['count_floors_pre_eq'] >= 2.1309) & (test_df['has_repair_started'] == 0),'buildings_require_immediate_attention_2']=1
train_df['buildings_require_immediate_attention_2'].fillna(0,inplace=True)
test_df['buildings_require_immediate_attention_2'].fillna(0,inplace=True)

# Buildings which were on geotechnical risk but stil not damaged
train_df.loc[(train_df['has_geotechnical_risk'] == 1) & (train_df['condition_post_eq'] == 'Not damaged'),'buildings_which_are_lucky']=1
test_df.loc[(test_df['has_geotechnical_risk'] == 1) & (test_df['condition_post_eq'] == 'Not damaged'),'buildings_which_are_lucky']=1
train_df['buildings_which_are_lucky'].fillna(0,inplace=True)
test_df['buildings_which_are_lucky'].fillna(0,inplace=True)

# Buildings which are at geotechnical risk but repair hasn't started yet
train_df.loc[(train_df['has_geotechnical_risk'] == 1) & (train_df['condition_post_eq'] == 'Not damaged') & (train_df['has_repair_started'] == 0),'govt_is_lazy']=1
test_df.loc[(test_df['has_geotechnical_risk'] == 1) & (test_df['condition_post_eq'] == 'Not damaged') & (test_df['has_repair_started'] == 0),'govt_is_lazy']=1
train_df['govt_is_lazy'].fillna(0,inplace=True)
test_df['govt_is_lazy'].fillna(0,inplace=True)

# Buildings with whose post count floors are more than pre count floors
cond_train_6=(train_df['difference_in_floors'] >= 0)
cond_test_6=(test_df['difference_in_floors'] >= 0)
train_df['mistake_in_counting_floors']=np.where(cond_train_6,1,0)
test_df['mistake_in_counting_floors']=np.where(cond_test_6,1,0)

# Buildings with whose post height is more than pre height
cond_train_7=(train_df['difference_in_height'] >= 0)
cond_test_7=(test_df['difference_in_height'] >= 0)
train_df['mistake_in_measuring_height']=np.where(cond_train_7,1,0)
test_df['mistake_in_measuring_height']=np.where(cond_test_7,1,0)

# Dividing Plinth area into categories
train_df.loc[(train_df['plinth_area_sq_ft']<391.312506),'plinth_area_sq_ft']=5
train_df.loc[(train_df['plinth_area_sq_ft']>391.312506) & (train_df['plinth_area_sq_ft']<398.505340),'plinth_area_sq_ft']=4
train_df.loc[(train_df['plinth_area_sq_ft']<419.486515) & (train_df['plinth_area_sq_ft']>398.505340),'plinth_area_sq_ft']=3
train_df.loc[(train_df['plinth_area_sq_ft']>419.486515) & (train_df['plinth_area_sq_ft']<451.417435),'plinth_area_sq_ft']=2
train_df.loc[(train_df['plinth_area_sq_ft']<507.869961) & (train_df['plinth_area_sq_ft']>451.417435),'plinth_area_sq_ft']=1
train_df.loc[(train_df['plinth_area_sq_ft']>507.869961),'plinth_area_sq_ft']=0

test_df.loc[(test_df['plinth_area_sq_ft']<391.312506),'plinth_area_sq_ft']=5
test_df.loc[(test_df['plinth_area_sq_ft']>391.312506) & (test_df['plinth_area_sq_ft']<398.505340),'plinth_area_sq_ft']=4
test_df.loc[(test_df['plinth_area_sq_ft']<419.486515) & (test_df['plinth_area_sq_ft']>398.505340),'plinth_area_sq_ft']=3
test_df.loc[(test_df['plinth_area_sq_ft']>419.486515) & (test_df['plinth_area_sq_ft']<451.417435),'plinth_area_sq_ft']=2
test_df.loc[(test_df['plinth_area_sq_ft']<507.869961) & (test_df['plinth_area_sq_ft']>451.417435),'plinth_area_sq_ft']=1
test_df.loc[(test_df['plinth_area_sq_ft']>507.869961),'plinth_area_sq_ft']=0

# Retaining Plinth area
train_df['plinth_area_sq_ft_new']=a
test_df['plinth_area_sq_ft_new']=b

# Dealing with the categorical columns

In [0]:
dummies=['area_assesed','district_id','legal_ownership_status','land_surface_condition','foundation_type','roof_type','ground_floor_type','other_floor_type'
        ,'position','plan_configuration','condition_post_eq']
for dummy in dummies:
  dummies_1 = pd.get_dummies(train_df.loc[:, dummy], prefix=dummy)  
  dummies_2 = pd.get_dummies(test_df.loc[:, dummy], prefix=dummy)  
  train_df = pd.concat([train_df, dummies_1], axis = 1)
  train_df = train_df.drop(dummy, axis =1)
  test_df = pd.concat([test_df, dummies_2], axis = 1)
  test_df = test_df.drop(dummy, axis =1)

# Making test file and train file ready

In [0]:
train_y=train_df['damage_grade'].values
test_id=test_df['building_id'].values
train_X=train_df.drop(['building_id','damage_grade','ward_id_x','ward_id_y','vdcmun_id','vdcmun_id_x','vdcmun_id_y','district_id_x','district_id_y'],axis=1)
test_X=test_df.drop(['building_id','ward_id_x','ward_id_y','vdcmun_id','vdcmun_id_x','vdcmun_id_y','district_id_x','district_id_y'],axis=1)

# Dropping unwanted features with the help of feature importance given by lightgbm

In [0]:
col_to_drop=['condition_post_eq_Damaged-Rubble Clear-New building built','condition_post_eq_Damaged-Rubble unclear','buildings_require_attention'
             ,'condition_post_eq_Damaged-Rubble clear','has_secondary_use_gov_office']
train_X=train_X.drop(col_to_drop,axis=1)
test_X=test_X.drop(col_to_drop,axis=1)

# Filling null values of test file
test_X['age_building_new'].fillna(27.805518,inplace=True)
test_X['age_building_new_2'].fillna(27.805518,inplace=True)
test_X['age_building_new_3'].fillna(27.805518,inplace=True)
test_X['age_building_new_4'].fillna(27.805518,inplace=True)
test_X['age_building_new_5'].fillna(27.805518,inplace=True)
test_X['condition-wise_age'].fillna(27.805518,inplace=True)

# Install the required libraries

In [12]:
!pip install lightgbm
import lightgbm as lgb

In [14]:
!pip install bayesian-optimization

  Running setup.py bdist_wheel for bayesian-optimization ... - done
  Stored in directory: /content/.cache/pip/wheels/f5/ef/5c/9ff1d2d86ad1117bf4cd3f989356ae46fa907014ece63b09dd
Successfully built bayesian-optimization


In [16]:
!pip install scikit-optimize
from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold, KFold

# Training lightgbm with scikit optimzation

In [0]:
def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest MSE: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__
    all_models.to_csv(clf_name+"_cv_results.csv")

In [0]:
bayes_cv_tuner = BayesSearchCV(
    estimator = lgb.LGBMClassifier(objective='multiclass', boosting_type='gbdt',
        learning_rate = 0.022066991249460103,
        sub_sample = 0.7048800081190388,
        num_leaves = 443,      
        max_depth = 20,
        colsample_bytree = 0.42149456283334996 ,
        subsample = 0.16628951140171316,
        min_child_samples = 26,
        max_bin= 1018,
        subsample_freq= 0,
        min_child_weight= 1,
        reg_lambda= 4.3806965488564525e-05,
        reg_alpha= 0.1611980387486336,
        scale_pos_weight= 0.0009365503147654213,
        n_estimators= 792,
                                  ),   
    search_spaces={'subsample_freq':(0,1)},
    scoring = 'f1_weighted', #neg_mean_squared_log_error
    cv = KFold(
        n_splits=6,
        shuffle=True,
        random_state=42
    ),
    n_jobs = 1,
    n_iter = 1,   
    verbose = 20,
    refit = True,
    random_state = 42
)


In [19]:
# Fit the model
result = bayes_cv_tuner.fit(train_X, train_y, callback=status_print)

Fitting 8 folds for each of 1 candidates, totalling 8 fits
[CV] subsample_freq=0 ................................................


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 23.0min remaining:    0.0s


[CV] ....... subsample_freq=0, score=0.7835320592203386, total=23.0min
[CV] subsample_freq=0 ................................................


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 45.4min remaining:    0.0s


[CV] ....... subsample_freq=0, score=0.7830141366727436, total=22.4min
[CV] subsample_freq=0 ................................................


...

[CV] ....... subsample_freq=0, score=0.7836501723224234, total=22.3min
[CV] subsample_freq=0 ................................................


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed: 158.9min remaining:    0.0s


[CV] ....... subsample_freq=0, score=0.7821757017430627, total=22.9min
[CV] subsample_freq=0 ................................................


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 181.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 181.4min finished


[CV] ....... subsample_freq=0, score=0.7816420500710872, total=22.6min
Model #1
Best MSE: 0.7834
Best params: {'subsample_freq': 0}



In [0]:
pred = bayes_cv_tuner.predict(test_X)
#pred_lgb = np.asarray([np.argmax(line) for line in pred])
pred

In [0]:
submit_df=pd.DataFrame({'building_id':test_id})
submit_df['damage_grade']=pred
p={0:'Grade 1',1:'Grade 2',2:'Grade 3',3:'Grade 4',4:'Grade 5'}
submit_df['damage_grade']=submit_df['damage_grade'].map(p)
submit_df['damage_grade'].head()

# Exporting the predictions to csv files

In [0]:
submit_df.to_csv('hackerearth_submission_141.csv',index=False) #75.453
from google.colab import files
downloaded=files.download('hackerearth_submission_141.csv') 